In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('diabetess.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [5]:
x = df.iloc[:, :-1].values
y = df.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(x)

In [7]:
X.shape

(768, 8)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [39]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [10]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = 8))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.4229 - loss: 0.7645 - val_accuracy: 0.5130 - val_loss: 0.7249
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5420 - loss: 0.6984 - val_accuracy: 0.6169 - val_loss: 0.6613
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6698 - loss: 0.6463 - val_accuracy: 0.7208 - val_loss: 0.6131
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6920 - loss: 0.6069 - val_accuracy: 0.7662 - val_loss: 0.5773
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6802 - loss: 0.5910 - val_accuracy: 0.7597 - val_loss: 0.5473
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7232 - loss: 0.5654 - val_accuracy: 0.7727 - val_loss: 0.5235
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7520 - loss: 0.5266 - val_accuracy: 0.7987 - val_loss: 0.5048
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7650 - loss: 0.5312 - val_accuracy: 0.8052 - 

# Further steps to learn

1. How to select appropriate optimizer
2. Number of nodes in a layer
3. how to select number of layers
4. All in all one model

In [12]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


In [13]:
import keras_tuner as kt

for tuner we have to build model function - build_model

hp - Hyperparameter

search more optimizers - https://keras.io/api/optimizers/

In [14]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32, activation = 'relu', input_dim = 8))
  model.add(Dense(1, activation = 'sigmoid'))
  # model.compile(optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'ramsprop', 'adadelta']), loss = 'binary_crossentropy', metrics = ['accuracy'])
  # another method
  optimizer = hp.Choice('optimizer', values = ['adam', 'sgd', 'ramsprop', 'adadelta'])
  model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model


**RandomSearch-** work like gridsearchcv in machine learning

In [15]:
tuner = kt.RandomSearch(build_model,
                        objective= 'val_accuracy',
                        max_trials = 5)

In [16]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 4 Complete [00h 00m 07s]
val_accuracy: 0.6558441519737244

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 15s


In [17]:
tuner.get_best_hyperparameters()[0].values # find activation function

{'optimizer': 'sgd'}

In [18]:
model = tuner.get_best_models(num_models = 1)[0] # extract top model using [0]

In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Retrain the model with best activation function
model.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7192 - loss: 0.5993 - val_accuracy: 0.7857 - val_loss: 0.5364
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7093 - loss: 0.5895 - val_accuracy: 0.7857 - val_loss: 0.5299
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7442 - loss: 0.5802 - val_accuracy: 0.7922 - val_loss: 0.5241
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7139 - loss: 0.5992 - val_accuracy: 0.7987 - val_loss: 0.5184
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7207 - loss: 0.5841 - val_accuracy: 0.7987 - val_loss: 0.5122
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7270 - loss: 0.5812 - val_accuracy: 0.7922 - val_loss: 0.5074
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7244 - loss: 0.5669 - val_accuracy: 0.7922 - val_loss: 0.5025
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7313 - loss: 0.5629 - val_accuracy: 0.7922 - 

# Find the right number of neuron or node in the layer

In [21]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units',min_value = 8, max_value = 128,step = 8)
  model.add(Dense(units, activation = 'relu', input_dim = 8))
  model.add(Dense(1, activation = 'sigmoid'))
  model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

In [22]:
tuner = kt.RandomSearch(build_model,
                        objective= 'val_accuracy',
                        max_trials = 5,
                        directory = 'mydir',
                        project_name = 'node_tuner')

In [23]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.8181818127632141

Best val_accuracy So Far: 0.8311688303947449
Total elapsed time: 00h 00m 14s


In [24]:
tuner.get_best_hyperparameters()[0].values

{'units': 96}

In [25]:
model = tuner.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [26]:
model.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7688 - loss: 0.4864 - val_accuracy: 0.8312 - val_loss: 0.4413
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7901 - loss: 0.4629 - val_accuracy: 0.8247 - val_loss: 0.4294
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7557 - loss: 0.4863 - val_accuracy: 0.8312 - val_loss: 0.4266
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7864 - loss: 0.4681 - val_accuracy: 0.8117 - val_loss: 0.4231
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7439 - loss: 0.4840 - val_accuracy: 0.8117 - val_loss: 0.4215
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7785 - loss: 0.4514 - val_accuracy: 0.8117 - val_loss: 0.4211
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7685 - loss: 0.4530 - val_accuracy: 0.8182 - val_loss: 0.4215
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7859 - loss: 0.4500 - val_accuracy: 0.8117 - 

# Select number of layers

In [27]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(72, activation = 'relu', input_dim = 8))
  for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
    model.add(Dense(72, activation = 'relu'))
  model.add(Dense(1, activation = 'sigmoid'))
  model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

In [28]:
tuner = kt.RandomSearch(build_model,
                        objective= 'val_accuracy',
                        max_trials = 5,
                        directory = 'mydir',
                        project_name = 'num_layers')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 5 Complete [00h 00m 06s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 27s


In [30]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [32]:
model.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8371 - loss: 0.3815 - val_accuracy: 0.8247 - val_loss: 0.4500
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8184 - loss: 0.3804 - val_accuracy: 0.8182 - val_loss: 0.4466
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8169 - loss: 0.4043 - val_accuracy: 0.8312 - val_loss: 0.4507
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8406 - loss: 0.3736 - val_accuracy: 0.8312 - val_loss: 0.4541
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8257 - loss: 0.3708 - val_accuracy: 0.8247 - val_loss: 0.4505
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8307 - loss: 0.3708 - val_accuracy: 0.8182 - val_loss: 0.4523
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8323 - loss: 0.3678 - val_accuracy: 0.8312 - val_loss: 0.4525
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8586 - loss: 0.3573 - val_accuracy: 0.8182 - v

In [34]:
def build_model(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
    if counter == 0:
      model.add(Dense(hp.Int('units' + str(i), min_value = 8, max_value = 128, step = 8),
                      activation = hp.Choice('activation' + str(i), values = ['relu', 'sigmoid', 'tanh']), input_dim = 8))

    else:
      model.add(Dense(hp.Int('units' + str(i), min_value = 8, max_value = 128, step = 8),
                      activation = hp.Choice('activation' + str(i), values = ['relu', 'sigmoid', 'tanh'])))
    counter += 1
  model.add(Dense(1, activation = 'sigmoid'))
  model.compile(optimizer = hp.Choice('optimizer', values = ['rmsprop','adam','sgd','nadam','adadelta']),
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])
  return model


In [35]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 3,
                        directory='mydir',
                        project_name = 'final')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 3 Complete [00h 00m 08s]
val_accuracy: 0.6948052048683167

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 20s


In [37]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10,
 'units0': 16,
 'activation0': 'tanh',
 'optimizer': 'adam',
 'units1': 96,
 'activation1': 'relu',
 'units2': 112,
 'activation2': 'tanh',
 'units3': 8,
 'activation3': 'tanh',
 'units4': 104,
 'activation4': 'tanh',
 'units5': 16,
 'activation5': 'tanh',
 'units6': 80,
 'activation6': 'tanh',
 'units7': 88,
 'activation7': 'tanh',
 'units8': 8,
 'activation8': 'relu',
 'units9': 8,
 'activation9': 'relu'}

In [ ]:
model = tuner.get_best_models(num_models = 1)[0]

In [38]:
model.fit(X_train, y_train, epochs = 200, initial_epoch=6, validation_data = (X_test, y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8665 - loss: 0.3375 - val_accuracy: 0.8117 - val_loss: 0.4811
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8556 - loss: 0.3499 - val_accuracy: 0.8117 - val_loss: 0.4762
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8680 - loss: 0.3307 - val_accuracy: 0.8182 - val_loss: 0.4777
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8374 - loss: 0.3577 - val_accuracy: 0.8182 - val_loss: 0.4793
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8707 - loss: 0.3109 - val_accuracy: 0.8182 - val_loss: 0.4760
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8563 - loss: 0.3316 - val_accuracy: 0.8117 - val_loss: 0.4812
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8463 - loss: 0.3253 - val_accuracy: 0.8182 - val_loss: 0.4789
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8405 - loss: 0.3399 - val_accuracy: 0.81

# Dropout

In [44]:
def build_model(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
    if counter == 0:
      model.add(Dense(hp.Int('units' + str(i), min_value = 8, max_value = 128, step = 8),
                      activation = hp.Choice('activation' + str(i), values = ['relu', 'sigmoid', 'tanh']), input_dim = 8))
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))

    else:
      model.add(Dense(hp.Int('units' + str(i), min_value = 8, max_value = 128, step = 8),
                      activation = hp.Choice('activation' + str(i), values = ['relu', 'sigmoid', 'tanh'])))
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
    counter += 1
  model.add(Dense(1, activation = 'sigmoid'))
  model.compile(optimizer = hp.Choice('optimizer', values = ['rmsprop','adam','sgd','nadam','adadelta']),
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])
  return model


In [45]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 3,
                        directory='mydir',
                        project_name = 'final1')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [46]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.6948052048683167

Best val_accuracy So Far: 0.7532467246055603
Total elapsed time: 00h 00m 11s


In [47]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7,
 'units0': 48,
 'activation0': 'tanh',
 'dropout0': 0.4,
 'optimizer': 'rmsprop',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1,
 'units5': 8,
 'activation5': 'relu',
 'dropout5': 0.1,
 'units6': 8,
 'activation6': 'relu',
 'dropout6': 0.1}

In [48]:
model = tuner.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [49]:
model.fit(X_train, y_train, epochs = 200, initial_epoch=6, validation_data = (X_test, y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.6907 - loss: 0.5897 - val_accuracy: 0.7987 - val_loss: 0.4738
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7474 - loss: 0.5526 - val_accuracy: 0.7922 - val_loss: 0.4626
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6894 - loss: 0.5566 - val_accuracy: 0.8052 - val_loss: 0.4536
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7143 - loss: 0.5441 - val_accuracy: 0.7857 - val_loss: 0.4480
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7288 - loss: 0.5073 - val_accuracy: 0.7987 - val_loss: 0.4433
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7669 - loss: 0.5384 - val_accuracy: 0.7922 - val_loss: 0.4399
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7518 - loss: 0.5366 - val_accuracy: 0.7987 - val_loss: 0.4467
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7446 - loss: 0.5111 - val_accuracy: 0.